## 实验题目2 龙贝格(Romberg)积分法

### 参考资料

julia 数值积分 https://blog.csdn.net/m0_37816922/article/details/103475445

julia SymPy.jl https://github.com/JuliaPy/SymPy.jl

SymPy.jl example https://docs.juliahub.com/SymPy/KzewI/1.0.29/introduction/

SymPy.jl assumption https://docs.sympy.org/dev/modules/core.html#module-sympy.core.assumptions

julia calculus https://docs.juliahub.com/CalculusWithJulia/AZHbv/0.0.16/


### 代码实现

In [175]:
using Printf
using Plots
using SymPy

In [176]:
function romberg(f::Function, xlim, iternum, ϵ)
    a, b = xlim
    n = iternum

    m = 1
    h = (b - a) / 2  # bug here
    T1 = 1 / 2 * h * (f(a) + f(b))
    @printf("\nT:%16.9f", T1)

    T2, C1, C2, S1, S2, R1, R2 = zeros(7)
    for i = 2:n
        ii = 2^(i - 1)
        tmpsum = 0.0
        for k = 1:ii
            tmpsum += f(a + (k - 1 / 2) * h)
        end
        # println("halfsum: $(tmpsum * h)")
        T2 = 1 / 2 * T1 + 1 / 2 * h * tmpsum
        @printf("\nT:%16.9f", T2)

        S2 = 1 / 3 * (4T2 - T1)
        @printf("\tS:%16.9f", S2)

        if m > 1
            C2 = 1 / 15 * (16S2 - S1)
            @printf("\tC:%16.9f", C2)
        end

        if m > 2
            R2 = 1 / 63 * (64C2 - C1)
            @printf("\tR:%16.9f", R2)
        end
        if m > 3
            tol = abs(R2 - R1)
            if tol < ϵ
                println("\nAccuracy requirement satisfied.")
                return
            end
        end
        # @label PARAM_UPDATE
        R1, C1, S1, T1 = R2, C2, S2, T2
        h /= 2
        m += 1
    end
    m
end


romberg (generic function with 1 method)

### 测试代码

#### Test 1 - Simple

In [177]:
# f(x) = x^2 * exp(x)
f(x) = 1/x
ϵ = 1e-6
xlim = 1, 3

romberg(f,xlim,10,ϵ)


T:     0.666666667
T:     0.866666667	S:     0.933333333
T:     0.978210678	S:     1.015392015	C:     1.020862594
T:     1.037267702	S:     1.056953376	C:     1.059724133	R:     1.060340983
T:     1.067651515	S:     1.077779453	C:     1.079167858	R:     1.079476489
T:     1.083059619	S:     1.088195653	C:     1.088890067	R:     1.089044387
T:     1.090817873	S:     1.093403957	C:     1.093751178	R:     1.093828338
T:     1.094710560	S:     1.096008122	C:     1.096181733	R:     1.096220313
T:     1.096660294	S:     1.097310205	C:     1.097397011	R:     1.097416301
T:     1.097636009	S:     1.097961247	C:     1.098004650	R:     1.098014295

10

In [178]:
# # compute definite integration
# @syms x y z
# f = 1 / x
# @time integrate(f, (x, 1, 3)) |> float

In [179]:
# f = 1 / (1 + x^4)
# @time integrate(f, (x, 1, 3)) |> float

### 实验题目

#### 问题 1

In [180]:
f(x) = x^2 * exp(x)
ϵ = 1e-6
xlim = 0, 1
iter_num = 30
romberg(f, xlim, iter_num, ϵ)

f(x) = exp(x)sin(x)
ϵ = 1e-6
xlim = 1, 3
romberg(f, xlim, iter_num, ϵ)


f(x) = 4 / (1 + x^2)
ϵ = 1e-6
xlim = 0, 1
romberg(f, xlim, iter_num, ϵ)

f(x) = 1 / (x + 1)
ϵ = 1e-6
xlim = 0, 1
romberg(f, xlim, iter_num, ϵ)


T:     0.679570457
T:     0.657551253	S:     0.650211518
T:     0.677367637	S:     0.683973099	C:     0.686223871
T:     0.695174509	S:     0.701110133	C:     0.702252602	R:     0.702507026
T:     0.706064798	S:     0.709694894	C:     0.710267211	R:     0.710394427
T:     0.712007419	S:     0.713988293	C:     0.714274520	R:     0.714338128
T:     0.715103147	S:     0.716135057	C:     0.716278174	R:     0.716309978
T:     0.716682119	S:     0.717208442	C:     0.717280001	R:     0.717295903
T:     0.717479381	S:     0.717745135	C:     0.717780915	R:     0.717788866
T:     0.717879957	S:     0.718013482	C:     0.718031372	R:     0.718035347
T:     0.718080731	S:     0.718147655	C:     0.718156600	R:     0.718158588
T:     0.718181239	S:     0.718214742	C:     0.718219214	R:     0.718220208
T:     0.718231524	S:     0.718248285	C:     0.718250521	R:     0.718251018
T:     0.718256674	S:     0.718265057	C:     0.718266175	R:     0.718266423
T:     0.718269250	S:     0.718273443	C:     0.71

### 思考题

2. 在实验 1 中二分次数和精度的关系如何？

   二分次数越多所求的精度越高，通常预设较大的二分次数来确保计算结果有足够的精度，同时也设定早停需要满足的精度要求，避免达到所需精度之后继续计算导致增加的运算量